In [2]:
import pandas as pd
import numpy as np
import datetime
import requests
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from requests.exceptions import ConnectionError
from bs4 import BeautifulSoup

In [4]:
# Obtiene todos los textos dentro de spans para un web_content especificado

def web_content_div(web_content,class_path):
    web_content_div = web_content.find_all('div',{'class': class_path})
    try:
        spans = web_content_div[0].find_all('fin-streamer')
        texts = [span.get_text() for span in spans]
    except IndexError:
        texts = []
    
    return texts

In [5]:
# Extrae la información de precio, % cambio y market cap para un código de stock
def real_time_price(stock_code):
   
    url='https://finance.yahoo.com/quote/'+ stock_code + '?p=' + stock_code + '&.tsrc=fin-srch'
    
    try:
        r = requests.get(url)
        web_content = BeautifulSoup(r.text, 'lxml')
        
        texts = web_content_div(web_content, "My(6px) Pos(r) smartphone_Mt(6px) W(100%)")
       
        if texts != []:
            price, change = texts[0], texts[2]
        else:
            price, change = [1], [1]
            print('Nothing found')
        data=pd.read_html(url)     
        market_cap=df=data[1].loc[0,1]
        
    except ConnectionError:
        price, change, market_cap = [], [], []
        print('Connection Error')
    return price, change, market_cap



In [6]:
# Extrae tabla de compañías del SP500 de wikipedia
sp500url='https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
code_table=pd.read_html(sp500url)
symbols = code_table[0]
symbols.rename(columns={'Symbol': 'stock_code', 'GICS Sector':'sector', 'GICS Sub-Industry':'industry'}, inplace=True)

,stock_code,Security,SEC filings,sector,industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [19]:
tickers=symbols['stock_code'].tolist()

True

In [20]:
# reemplaza tickers con '.'

for i in range(len(tickers)):
    if tickers[i]=='BRK.B':
        tickers[i]='BRK-B'
    elif tickers[i]=='BF.B':
        tickers[i]='BF-B'

In [23]:
# Aplica el webscrapping para una lista de tickers

tickers = ['AAPL', 'GOOGL','ABBV', 'AMZN', 'TSLA', 'MSFT', 'JNJ', 'HON', 'ETN', 'XOM', 'COP']
info = []

for stock_code in tickers:
    time_stamp = datetime.datetime.now()
    time_stamp = time_stamp.strftime('%d-%m-%Y %H:%M:%S')
    price, change, market_cap = real_time_price(stock_code)
    
    info.append(
        {
            'time_stamp': time_stamp,
            'stock_code': stock_code,
            'price': price,
            'change': change,
            'market_cap': market_cap
        }  
    )
prices_df=pd.DataFrame(info)

In [24]:
# Join con la tabla de symbols
df=pd.merge(prices_df, symbols[['stock_code','sector', 'industry']], on='stock_code', how='left')
df['change']=df['change'].str.replace('[()]','')



C:\Users\filip\AppData\Local\Temp\ipykernel_18816\911075935.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['change']=df['change'].str.replace('[()]','')


In [25]:
df['change']=(df['change'].str.rstrip('%').astype(float))
df['price']=df['price'].astype(float)

# convierte market_cap de str a int en billions
multipliers = {'K':1000, 'M':1000000, 'B':1000000000, 'T':1000000000000}
def string_to_billions(string):
    if string[-1].isdigit():
        return float(string)/1000000000
    mult =multipliers[string[-1]]
    return float(string[:-1])*mult/1000000000

df['market_cap'] = df['market_cap'].apply(string_to_billions)
df

,time_stamp,stock_code,price,change,market_cap,sector,industry
0,05-12-2022 13:48:49,AAPL,146.73,-0.73,2335.000,Information Technology,"Technology Hardware, Storage & Peripherals"
1,05-12-2022 13:48:52,GOOGL,100.20,-0.24,1299.000,Communication Services,Interactive Media & Services
2,05-12-2022 13:48:55,ABBV,163.86,0.12,289.783,Health Care,Pharmaceuticals
3,05-12-2022 13:48:58,AMZN,92.07,-2.19,939.271,Consumer Discretionary,Internet & Direct Marketing Retail
4,05-12-2022 13:49:01,TSLA,184.72,-5.20,583.334,Consumer Discretionary,Automobile Manufacturers
5,05-12-2022 13:49:03,MSFT,251.34,-1.44,1874.000,Information Technology,Systems Software
6,05-12-2022 13:49:05,JNJ,179.09,0.12,468.221,Health Care,Pharmaceuticals
7,05-12-2022 13:49:08,HON,216.74,-1.07,145.719,Industrials,Industrial Conglomerates
8,05-12-2022 13:49:11,ETN,160.99,-1.42,64.024,Industrials,Electrical Components & Equipment
9,05-12-2022 13:49:14,XOM,108.44,-1.29,446.587,Energy,Integrated Oil & Gas


In [27]:
fig = px.treemap(df, path=['sector', 'industry', 'stock_code'], values='market_cap',
                  color='change', hover_data=['change'], 
                  color_continuous_scale=[(0, "#f63538"), (0.5, "#414554"), (1, "#30cc5a")],
                  color_continuous_midpoint=0, range_color=[-3,3])

fig.data[0].customdata = fig.data[0].marker.colors
fig.data[0].texttemplate = "<b>%{label}</b><br>Market Cap: %{value:.0f} B<br>Change: %{customdata:.2f}%<br>"
fig.update_traces(hovertemplate='%{label}<br>Change=%{customdata:.2f}%')
fig.update_traces(marker=dict(line=dict(width=1, color='#262931')))
fig.update_layout({
    'plot_bgcolor': 'rgba(0,0,0,0)',
    'paper_bgcolor': 'rgba(0,0,0,0)'
})
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25), autosize=True)
fig.show()